## Might be usefull later, but for now useless

## Imports

In [34]:
from itertools import permutations

In [35]:
import random

## Vars

In [69]:
player_options = [
        ["X",None,None,
         None,None,None,
         None,None,None],

         [None,"X",None,
          None,None,None,
          None,None,None],

          [None,None,"X",
          None,None,None,
          None,None,None],

          [None,None,None,
          "X",None,None,
          None,None,None],

          [None,None,None,
          None,"X",None,
          None,None,None],

          [None,None,None,
          None,None,"X",
          None,None,None],

          [None,None,None,
          None,None,None,
          "X",None,None],

          [None,None,None,
          None,None,None,
          None,"X",None],

          [None,None,None,
          None,None,None,
          None,None,"X"]
]

In [70]:
class State:
  def __init__(self, env, value, parent, children):
    self.parent = parent
    self.children = children
    self.env = env
    self.value = value

In [71]:
# Root state
root_state = State([None,None,None,
                   None,None,None,
                   None,None,None], 
                   None,
                   None,
                   [])

curr_state = root_state

## Functions

In [39]:
# If exist in the tree, return it, if not, add it
def get_new_state(state, action):
  new_state = State([],0,state,[])

  if action == None:
    return state
  
  # Set the new state env after action
  for state_action in zip(state.env,action):
    if(state_action[0] == None and state_action[1] != None):
      new_state.env.append(state_action[1])
    else:
      new_state.env.append(state_action[0])

  # If this state env already in tree, return it
  if state.children != None:
    for child_state in state.children:
      if child_state.env == new_state.env:
        return child_state
  
  # If not, append this new state to tree and return it
  new_state.parent = state
  state.children.append(new_state)
  return new_state 

In [40]:
def player_turn(state, action):
  return get_new_state(state,action)

In [41]:
# Takes state and returns an action
def agent_turn(state):

  # Determine possible actions
  possible_actions = []
  for i in range(0,len(state.env)):
    if state.env[i] == None:
      new_action = [None] * 9
      new_action[i] = "O"
      possible_actions.append(new_action)

  # Exploit or Expore
  # For now, we will always exploit, unless we have not seen this state
  # If we have not seen this state, we choose randomly, e.g. Explore
  exploit = True
  explore = True

  # Exploit: refer to seen states, choose action that yields highest value state
  if exploit:
    best_action = possible_actions[0]
    best_value_state = get_new_state(state,best_action)

    for action in possible_actions[1:]: 
      new_possible_state = get_new_state(state,action)
      
      if new_possible_state.value > best_value_state.value:
        best_action = action
      
      best_value_state = get_new_state(state,best_action)
  
  if explore:
    # Explore: select from possible actions randomly
    best_action = random.choice(possible_actions)

  # Make the play
  return get_new_state(state,best_action)

In [42]:
def check_for_win(state):
  # Horizontal lines
  if state.env[0] == state.env[1] == state.env[2] != None:
    return state.env[1]
  if state.env[3] == state.env[4] == state.env[5] != None:
    return state.env[4]
  if state.env[6] == state.env[7] == state.env[8] != None:
    return state.env[7]
  # Vertical lines
  if state.env[0] == state.env[3] == state.env[6] != None:
    return state.env[3]
  if state.env[1] == state.env[4] == state.env[7] != None:
    return state.env[4]
  if state.env[2] == state.env[5] == state.env[8] != None:
    return state.env[5]
  # Diagonals
  if state.env[0] == state.env[4] == state.env[8] != None:
    return state.env[4]
  if state.env[2] == state.env[4] == state.env[6] != None:
    return state.env[4]

In [43]:
step_size = 0.1
def value_function(state):
  return step_size*(state.value - state.parent.value)

In [44]:
def back_propagate(state):
  state_iter = state
  while state_iter.parent.value != None:
    state_iter.parent.value = value_function(state_iter) # this should be a value function, not 0.1
    state_iter = state_iter.parent

## Execution

In [72]:
# Gameloop

while (1):
  print("\nyour turn:")
  player_choice = input("\n0 | 1 | 2\n3 | 4 | 5\n6 | 7 | 8\n\n")

  curr_state = player_turn(curr_state,player_options[int(player_choice)])

  print(curr_state.env[0], " | ", curr_state.env[1], " | ", curr_state.env[2])
  print(curr_state.env[3], " | ", curr_state.env[4], " | ", curr_state.env[5])
  print(curr_state.env[6], " | ", curr_state.env[7], " | ", curr_state.env[8])

  if check_for_win(curr_state) == "X":
    curr_state.value = -1 # This was a loss for the agent
    back_propagate(curr_state)
    break
    
  print("\nagent's turn...")
  curr_state = agent_turn(curr_state)

  print(curr_state.env[0], " | ", curr_state.env[1], " | ", curr_state.env[2])
  print(curr_state.env[3], " | ", curr_state.env[4], " | ", curr_state.env[5])
  print(curr_state.env[6], " | ", curr_state.env[7], " | ", curr_state.env[8])

  if check_for_win(curr_state) == "O":
    curr_state.value = 1 # This was a win for the agent
    back_propagate(curr_state)
    break



your turn:

0 | 1 | 2
3 | 4 | 5
6 | 7 | 8

4
None  |  None  |  None
None  |  X  |  None
None  |  None  |  None

agent's turn...
None  |  None  |  None
None  |  X  |  O
None  |  None  |  None

your turn:

0 | 1 | 2
3 | 4 | 5
6 | 7 | 8

2
None  |  None  |  X
None  |  X  |  O
None  |  None  |  None

agent's turn...
None  |  None  |  X
None  |  X  |  O
None  |  O  |  None

your turn:

0 | 1 | 2
3 | 4 | 5
6 | 7 | 8

6
None  |  None  |  X
None  |  X  |  O
X  |  O  |  None


## Visualizations

In [73]:
# Print backpropagated path
state_iter = curr_state
while state_iter.value != None:
    print(state_iter.env)
    print(state_iter.value)
    state_iter = state_iter.parent

[None, None, 'X', None, 'X', 'O', 'X', 'O', None]
-1
[None, None, 'X', None, 'X', 'O', None, 'O', None]
-0.1
[None, None, 'X', None, 'X', 'O', None, None, None]
-0.010000000000000002
[None, None, None, None, 'X', 'O', None, None, None]
-0.0010000000000000002
[None, None, None, None, 'X', None, None, None, None]
-0.00010000000000000003


In [ ]:
# TODO: actually traverse this properly,
# currently only traverses throught the first child of every node
# Print current state tree
state_iter = root_state
while len(state_iter.children) != 0:
    print("State: ", state_iter.env, " : ", state_iter.value)
    print("Children:")
    for child in state_iter.children:
      print(child.env)

    if len(state_iter.children)!= 0:
      state_iter = state_iter.children[0]